In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
import pickle
from sklearn.ensemble import RandomForestClassifier as RFC
    



/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#lodaing data
data_path = "/home/raku/kaggleData/2sigma/xgb142/"
store = "/home/raku/kaggleData/2sigma/rf2000/"
train_file = data_path + "xgb1.42-train.json"
test_file = data_path + "xgb1.42-test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

feature_file = data_path+'xgb142features.pickle'
fileObject = open(feature_file,'r') 
features = pickle.load(fileObject)
fileObject.close()

(49352, 293)
(74659, 292)


In [3]:
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)



In [6]:
cv_scores=[]
i=0


In [5]:
for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    rf = RFC(2000)
    rf.fit(dev_X,dev_y)
    preds = rf.predict_proba(val_X)

    #save the pickles for futures use
    pickl_file = store+'rf2000-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 0.569642903575
loss for the turn 2 is 0.564160921334
loss for the turn 3 is 0.565056791973
loss for the turn 4 is 0.565343447361
loss for the turn 5 is 0.563333790506
The mean of the cv_scores is:
0.56550757095


In [7]:
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

rf = RFC(2000)
rf.fit(train_X,train_y)
preds = rf.predict_proba(test_X)

out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'rf2000-bulk-out.json')